In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import confusion_matrix, silhouette_score
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from yellowbrick.classifier import ConfusionMatrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier)
from sklearn.pipeline import make_pipeline

from IPython.display import display, HTML
pd.options.display.max_columns=None

In [2]:
stock_data=pd.read_pickle("process_data/clean_data/sandp_ta.plk")

In [22]:
os.chdir(r"C:\Users\power\Dropbox\Cory")

In [3]:
#clean quarterly financial data

quarterly_fin=pd.read_csv("SP500_quarterly_fin.zip")

#vars to deal with

date_vars=['datadate','fyearq','fqtr','fyr']

num_vars=['acchgq','acoq','actq','altoq','ancq','anoq',\
'aociderglq','aociotherq','aocisecglq','aol2q','aoq','apq','aqaq',\
'aqdq','aqepsq','aqpl1q','aqpq','arcedq','arceepsq','arceq','atq',\
'aul3q','billexceq','capr1q','capr2q','capsftq','capsq','capxy',\
'cdvcy','chechy','ceiexbillq','ceqq','cheq','chq','cibegniq','cicurrq',\
'ciderglq','cimiiq','ciotherq','ciy','cogsq','cshfd12','cshfdq','cshiq',\
'cshopq','cshoq','cshprq','cshtrq','cstkeq','cstkq','dcomq','dd1q','deracq',\
'deraltq','derhedglq','derlcq','derlltq','diladq','dlcq','dlttq','doq',\
'dpacreq','dpactq','dpq','dpretq','drcq','drltq','dteaq','dtedq','dteepsq',\
'dtepq','dvintfq','dvpq','dvpspq','epsfiq','epsfxy','esoptq','esubq','fcaq',\
'ffoq','finacoq','finchq','findlcq','findltq','finivstq','finltoq','finreccq',\
'finrecltq','finrevq','finxintq','finxoprq','gdwlamq','gdwliaq','gdwlidq',\
'gdwliepsq','gdwlipq','gdwlq','glaq','glceaq','glceepsq','glcepq','glivq',\
'glpq', 'hedgeglq','ibq','icaptq','intaccq','intanoq','intanq','invfgq','invtq',\
'itccy','ivaeqq','ivltq','ivstq','lctq','lltq','lnoq','lol2q','lul3q','ltq','mibnq',\
'mibq','mibtq','miiq','mkvaltq','msaq','ncoq','niitq','nimq','niq','nopiq','npatq',\
'npq','nrtxtepsq','nrtxtq','obkq','oepsxq','oiadpq','opepsq','piq','pllq',\
'pncepsq','pncpq','pstkq','rcaq','rcdq','recdq','recdq','rdipq','rectq','req',\
'revtq','rllq','rraq','saleq','setaq','spceq','spiq','tiiq','tieq','txdbaq','txdbclq',\
'txdbq','txwq','uaptq','uceqq','wcapq','wdaq','xaccq','xiq']

cat_vars=['gvkey','tic','cusip','idbflag','loc','naics','state']

#drop those missing columns
quarterly_info=quarterly_fin[num_vars].describe().T
num_filtered=list(quarterly_info[quarterly_info['count']>13000].index)

#data filtered
quarterly_fin_filtered=quarterly_fin[date_vars+num_filtered+cat_vars]
quarterly_fin_filtered.loc[quarterly_fin_filtered['state'].isnull(),'state']=quarterly_fin_filtered['loc']
quarterly_fin_filtered.fillna(0, inplace=True)
quarterly_fin_filtered['year']=np.floor(quarterly_fin_filtered['datadate']/10000)
quarterly_fin_filtered['quarter']=np.floor((quarterly_fin_filtered['datadate']-quarterly_fin_filtered['year']*10000)/100)//3+quarterly_fin_filtered['year']*100

#dealing with dummies
states=pd.get_dummies(quarterly_fin_filtered['state'])
states.columns=["state_"+str(i) for i in states.columns]

naics=pd.get_dummies(quarterly_fin_filtered['naics'])
naics.columns=["naics_"+str(i) for i in naics.columns]

idbflag=pd.get_dummies(quarterly_fin_filtered['idbflag'])
idbflag.columns=["idbflag_"+str(i) for i in idbflag.columns]

#save the data
quarterly_fin_filtered=pd.concat([quarterly_fin_filtered, states, naics,idbflag], axis=1).drop(['idbflag','loc','naics','state'], axis=1)

quarterly_fin_filtered.to_pickle("quarterly_spy.pkl")

In [6]:
#deal with institutional ownership data
institutional=pd.read_csv("stock_institutional_ownership.csv", encoding='latin1')

institutional_vars=['quarter','TIC','mktcap','io_usd', 'nbr_firms', 'io', \
                    'io_dom', 'io_for', 'io_for_us','io_for_nus', 'io_dom_indep', \
                    'io_dom_grey', 'io_for_indep',\
                    'io_for_grey', 'io_cat1', 'io_cat2', 'io_cat3', 'io_cat4', 'io_cat5',\
                    'io_cat6', 'io_cat7', 'io_indep', 'io_grey', 'io_common', 'io_civil',\
                    'io_for_common', 'io_for_civil', 'ibh_5pct', 'ibh_1pct', 'top5','herf']

institutional_filtered=institutional[institutional_vars]

institutional_filtered.rename(columns={'TIC':'tic'}, inplace=True)

institutional_filtered.drop_duplicates(['tic','quarter'], inplace=True)

#merge with financial data
fin_and_inst=quarterly_fin_filtered.merge(institutional_filtered, on=['tic','quarter'])

fin_and_inst.to_pickle("quarterly_spy_w_inst.pickle")

['.dropbox',
 'company_financial_info.zip',
 'desktop.ini',
 'execucomp',
 'Execucomp_Data_Definitions.pdf',
 'Fundamentals Quarterly Manual.html',
 'Fundamentals Quarterly Manual_files',
 'new.pkl',
 'quarterly_spy.pkl',
 'quarterly_spy_w_inst',
 'quarterly_spy_w_inst.pickle',
 'Security Daily Manual.html',
 'Security Daily Manual_files',
 'SP500_daily.zip',
 'SP500_quarterly_fin.zip',
 'stock_institutional_ownership.csv',
 'ticker_prices.zip']

In [20]:
#merge everything with stock prices

quarterly_fin_filtered=pd.read_pickle("quarterly_spy_w_inst.pickle")

quarterly_fin_filtered['date']=pd.to_datetime(quarterly_fin_filtered['datadate'].astype(str),format='%Y%m%d')
quarterly_fin_filtered.set_index(['date'], inplace=True)
quarterly_fin_to_merge=quarterly_fin_filtered.groupby('tic').apply(lambda x: x.asfreq("D"))
merged=stock_data.drop('tic',axis=1).join(quarterly_fin_filled.drop('tic',axis=1), how='left')

#since the frequency is not the same across all the data, after merge, we ffill na values
merged=merged.groupby('tic', as_index=False).fillna(method='ffill')
merged=merged.reset_index()
merged_filled.dropna(inplace=True)

merged_filled.to_pickle("new.pkl")


KeyboardInterrupt



### some trial code for technical notebook

In [64]:
os.chdir(r"C:\Users\power\Dropbox\Cory")

In [65]:
#preprocessing and rescaling

full_data=pd.read_pickle("full_data.pkl")

X = full_data.drop('roi_class', axis=1)
y = full_data.roi_class

X_poly=pd.concat([X.loc[:,'30 period CCI_30':"MFV_"], X.loc[:,'30 period CCI_30_sandp':]], axis=1)
X_dummy=X.loc[:,'state_AL':'idbflag_D']
X_normal=X.drop((list(X_poly.columns)+list(X_dummy.columns)), axis=1)

sd = StandardScaler()

poly = PolynomialFeatures(degree=3)
Xp = poly.fit_transform(X_poly)
X_to_scale=pd.concat([pd.DataFrame(Xp),X_Normal], axis=1)
X_scaled=sd.fit_transform(X_to_scale)
X_joined=pd.concat([pd.DataFrame(X_scaled), X_dummy], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_joined, y, test_size=0.2)
X_train, X_train_lr, y_train, y_train_lr = train_test_split(
    X_train, y_train, test_size=0.2)

#### Random Forest and logit ensembling and SVM

In [73]:
rf = RandomForestClassifier(max_depth=7, n_estimators=300)
rf.fit(X_train, y_train)
rf_lm = LogisticRegression(solver='lbfgs')
rf_enc = OneHotEncoder(categories='auto')
rf.fit(X_train, y_train)
rf_enc.fit(rf.apply(X_train))
rf_lm.fit(rf_enc.transform(rf.apply(X_train_lr)), y_train_lr)

RF_Logit_ensemble_test = rf_lm.score(rf_enc.transform(rf.apply(X_test)),y_test)
RF_Logit_ensemble_train = rf_lm.score(rf_enc.transform(rf.apply(X_train)),y_train)
print(RF_Logit_ensemble_train, RF_Logit_ensemble_test)

#deal with svc
gammas = [0.1, 0.5, 1, 5, 10]
score_dict={}
for gamma in gammas:
   svc = SVC(kernel='rbf', gamma=gamma).fit(X_train, y_train)
   score_dict[gamma]=(y_test, svc.predict(X_test))


C:\Users\power\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\power\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\power\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\power\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

#### some grid search cross validation for PCA

In [ ]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
logistic = SGDClassifier(loss='log', penalty='l2', early_stopping=True,
                         max_iter=10000, tol=1e-5, random_state=0)
pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

# Parameters of pipelines can be set using '__' separated parameter names:
param_grid = {
    'pca__n_components': [50, 100, 150, 200, 300, 400, 600],
    'logistic__alpha': np.logspace(-10, 10, 5),
}

search = GridSearchCV(pipe, param_grid, iid=False, cv=3)
search.fit(X_joined, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

# Plot the PCA spectrum
pca.fit(X_joined)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(pca.explained_variance_ratio_, linewidth=2)
ax0.set_ylabel('PCA explained variance')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
ax0.legend(prop=dict(size=12))

# For each number of components, find the best classifier results
results = pd.DataFrame(search.cv_results_)
components_col = 'param_pca__n_components'
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, 'mean_test_score'))

best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',
               legend=False, ax=ax1)
ax1.set_ylabel('Classification accuracy (val)')
ax1.set_xlabel('n_components')

plt.tight_layout()
plt.show()